In [27]:
%reload_ext autoreload
%autoreload 2

In [28]:
from fastai.basics import *

# Rossmann

## Data preparation / Feature engineering

In addition to the provided data, we will be using external datasets put together by participants in the Kaggle competition. You can download all of them [here](http://files.fast.ai/part2/lesson14/rossmann.tgz). Then you shold untar them in the dirctory to which `PATH` is pointing below.

For completeness, the implementation used to put them together is included below.

In [29]:
Config.DEFAULT_CONFIG_PATH = 'd:/fastaiconfig.yml'
PATH=Config().data_path()/Path('rossmann/')
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(PATH/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train),len(test), train.StateHoliday[1], type(train.StateHoliday[1])

(1017209, 41088, '0', str)

We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [30]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'
train.StateHoliday

0          False
1          False
2          False
3          False
4          False
           ...  
1017204     True
1017205     True
1017206     True
1017207     True
1017208     True
Name: StateHoliday, Length: 1017209, dtype: bool

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [31]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

Join weather/state names.

In [32]:
weather = join_df(weather, state_names, "file", "StateName")
weather

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15835,Saarland,2015-09-13,21,17,12,16,14,12,100,88,...,10.0,27,10,50.0,1.02,7.0,Rain,113,Saarland,SL
15836,Saarland,2015-09-14,18,14,11,15,12,7,99,85,...,4.0,32,16,53.0,9.91,5.0,Rain-Thunderstorm,213,Saarland,SL
15837,Saarland,2015-09-15,16,12,9,11,8,7,93,77,...,10.0,34,14,NaN,0.00,5.0,Rain,193,Saarland,SL
15838,Saarland,2015-09-16,19,15,11,16,13,10,97,90,...,4.0,32,14,45.0,20.07,7.0,Rain-Thunderstorm,147,Saarland,SL


In pandas you can add new columns to a dataframe by simply defining it. We'll do this for googletrends by extracting dates and state names from the given data and adding those columns.

We're also going to replace all instances of state name 'NI' to match the usage in the rest of the data: 'HB,NI'. This is a good opportunity to highlight pandas indexing. We can use `.loc[rows, cols]` to select a list of rows and a list of columns from the dataframe. In this case, we're selecting rows w/ statename 'NI' by using a boolean list `googletrend.State=='NI'` and selecting "State".

In [33]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
a = googletrend.copy()
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
b = googletrend.copy()
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'
c = googletrend.copy()
a,b,c

(                file                     week  trend        Date
 0     Rossmann_DE_SN  2012-12-02 - 2012-12-08     96  2012-12-02
 1     Rossmann_DE_SN  2012-12-09 - 2012-12-15     95  2012-12-09
 2     Rossmann_DE_SN  2012-12-16 - 2012-12-22     91  2012-12-16
 3     Rossmann_DE_SN  2012-12-23 - 2012-12-29     48  2012-12-23
 4     Rossmann_DE_SN  2012-12-30 - 2013-01-05     67  2012-12-30
 ...              ...                      ...    ...         ...
 2067  Rossmann_DE_SL  2015-08-30 - 2015-09-05     95  2015-08-30
 2068  Rossmann_DE_SL  2015-09-06 - 2015-09-12     47  2015-09-06
 2069  Rossmann_DE_SL  2015-09-13 - 2015-09-19     80  2015-09-13
 2070  Rossmann_DE_SL  2015-09-20 - 2015-09-26     57  2015-09-20
 2071  Rossmann_DE_SL  2015-09-27 - 2015-10-03      0  2015-09-27
 
 [2072 rows x 4 columns],
                 file                     week  trend        Date State
 0     Rossmann_DE_SN  2012-12-02 - 2012-12-08     96  2012-12-02    SN
 1     Rossmann_DE_SN  2012-12-09 - 

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [34]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [42]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)
googletrend

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,Rossmann_DE_SL,2015-08-30 - 2015-09-05,95,2015-08-30,SL,2015,8,35,30,6,242,False,False,False,False,False,False,1440892800
2068,Rossmann_DE_SL,2015-09-06 - 2015-09-12,47,2015-09-06,SL,2015,9,36,6,6,249,False,False,False,False,False,False,1441497600
2069,Rossmann_DE_SL,2015-09-13 - 2015-09-19,80,2015-09-13,SL,2015,9,37,13,6,256,False,False,False,False,False,False,1442102400
2070,Rossmann_DE_SL,2015-09-20 - 2015-09-26,57,2015-09-20,SL,2015,9,38,20,6,263,False,False,False,False,False,False,1442707200


The Google trends data has a special category for the whole of the Germany - we'll pull that out so we can use it explicitly.

In [44]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']
trend_de

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
148,Rossmann_DE,2012-12-02 - 2012-12-08,77,2012-12-02,None,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
149,Rossmann_DE,2012-12-09 - 2012-12-15,76,2012-12-09,None,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
150,Rossmann_DE,2012-12-16 - 2012-12-22,85,2012-12-16,None,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
151,Rossmann_DE,2012-12-23 - 2012-12-29,59,2012-12-23,None,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
152,Rossmann_DE,2012-12-30 - 2013-01-05,61,2012-12-30,None,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,Rossmann_DE,2015-08-30 - 2015-09-05,78,2015-08-30,None,2015,8,35,30,6,242,False,False,False,False,False,False,1440892800
292,Rossmann_DE,2015-09-06 - 2015-09-12,71,2015-09-06,None,2015,9,36,6,6,249,False,False,False,False,False,False,1441497600
293,Rossmann_DE,2015-09-13 - 2015-09-19,77,2015-09-13,None,2015,9,37,13,6,256,False,False,False,False,False,False,1442102400
294,Rossmann_DE,2015-09-20 - 2015-09-26,67,2015-09-20,None,2015,9,38,20,6,263,False,False,False,False,False,False,1442707200


Now we can outer join all of our data into a single dataframe. Recall that in outer joins everytime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has Null values for all right table fields. One way to check that all records are consistent and complete is to check for Null values post-join, as we do here.

*Aside*: Why not just do an inner join?
If you are assuming that all records are complete and match on the field you desire, an inner join will do the same thing as an outer join. However, in the event you are wrong or a mistake is made, an outer join followed by a null-check will catch it. (Comparing before/after # of rows for inner join is equivalent, but requires keeping track of before/after row #'s. Outer join is easier.)

In [45]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [46]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [47]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [48]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [49]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [50]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)


In [52]:
joined,joined_test

(         Store  DayOfWeek       Date  Sales  Customers  Open  Promo  \
 0            1          5 2015-07-31   5263        555     1      1   
 1            2          5 2015-07-31   6064        625     1      1   
 2            3          5 2015-07-31   8314        821     1      1   
 3            4          5 2015-07-31  13995       1498     1      1   
 4            5          5 2015-07-31   4822        559     1      1   
 ...        ...        ...        ...    ...        ...   ...    ...   
 1017204   1111          2 2013-01-01      0          0     0      0   
 1017205   1112          2 2013-01-01      0          0     0      0   
 1017206   1113          2 2013-01-01      0          0     0      0   
 1017207   1114          2 2013-01-01      0          0     0      0   
 1017208   1115          2 2013-01-01      0          0     0      0   
 
          StateHoliday  SchoolHoliday  Year  ...  Mean_VisibilityKm  \
 0               False              1  2015  ...               

Next we'll fill in missing values to avoid complications with `NA`'s. `NA` (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.

In [51]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

Next we'll extract features "CompetitionOpenSince" and "CompetitionDaysOpen". Note the use of `apply()` in mapping a function across dataframe values.

In [53]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

We'll replace some erroneous / outlying data.

In [54]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [55]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5], dtype=int64)

Same process for Promo dates. You may need to install the `isoweek` package first.

In [57]:
# If needed, uncomment:
! pip install isoweek

In [58]:
from isoweek import Week
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [59]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [60]:
joined.to_pickle(PATH/'joined')
joined_test.to_pickle(PATH/'joined_test')

## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [61]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

We'll be applying this to a subset of columns:

In [62]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [63]:
#df = train[columns]
df = train[columns].append(test[columns])

Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call `add_elapsed('SchoolHoliday', 'After')`:
This will apply to each row with School Holiday:
* A applied to every row of the dataframe in order of store and date
* Will add to the dataframe the days since seeing a School Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [66]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We'll do this for two more fields.

In [64]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [65]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We're going to set the active index to Date.

In [67]:
df = df.set_index("Date")

Then set null values from elapsed field calculations to 0.

In [68]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [69]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

In [70]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [71]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

Next we want to drop the Store indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [72]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [73]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [74]:
df.reset_index(inplace=True)

Now we'll merge these values onto the df.

In [75]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [76]:
df.drop(columns,1,inplace=True)

In [77]:
df.head()

,Date,Store,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,AfterSchoolHoliday,BeforeSchoolHoliday,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,105,0,0,0,13,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,104,0,0,0,12,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,103,0,0,0,11,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,102,0,0,0,10,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,101,0,9,-1,9,0,0.0,0.0,0.0,0.0,0.0,4.0


It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [78]:
df.to_pickle(PATH/'df')

In [79]:
df["Date"] = pd.to_datetime(df.Date)

In [80]:
df.columns

Index(['Date', 'Store', 'AfterStateHoliday', 'BeforeStateHoliday',
       'AfterPromo', 'BeforePromo', 'AfterSchoolHoliday',
       'BeforeSchoolHoliday', 'SchoolHoliday_bw', 'StateHoliday_bw',
       'Promo_bw', 'SchoolHoliday_fw', 'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [81]:
joined = pd.read_pickle(PATH/'joined')
joined_test = pd.read_pickle(PATH/f'joined_test')

In [83]:
joined = join_df(joined, df, ['Store', 'Date'])

In [84]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

The authors also removed all instances where the store had zero sale / was closed. We speculate that this may have cost them a higher standing in the competition. One reason this may be the case is that a little exploratory data analysis reveals that there are often periods where stores are closed, typically for refurbishment. Before and after these periods, there are naturally spikes in sales that one might expect. By ommitting this data from their training, the authors gave up the ability to leverage information about these periods to predict this otherwise volatile behavior.

In [85]:
joined = joined[joined.Sales!=0]

We'll back this up as well.

In [86]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [87]:
joined.to_pickle(PATH/'train_clean')
joined_test.to_pickle(PATH/'test_clean')